In [1]:
import os
os.environ["PYTHON_EXEC"] = "/Users/jessieyu/.pyenv/versions/qka-demo/bin/python3"
os.environ["NTC_DOC_FILE"] = "runtime/program_doc.json"
os.environ["NTC_PROGRAM_FILE"] = "runtime/qka_runtime_outer.py"

# List available quantum programs

In [2]:
from qiskit import IBMQ

provider = IBMQ.load_account()
provider.runtime.programs()

"Quantum-Kernel-Alignment":
  Description: Quantum kernel alignment algorithm that learns, on a given dataset, a quantum kernel maximizing the SVM classification margin.
  Parameters:
    - feature_map:
      description: An instance of FeatureMapForrelation in dictionary format used to map classical data into a quantum state space.
      type: dict
    - data:
      description: NxD array of training data, where N is the number of samples and D is the feature dimension.
      type: numpy.ndarray
    - labels:
      description: Nx1 array of +/-1 labels of the N training samples.
      type: numpy.ndarray
    - lambda_initial:
      description: Initial parameters of the quantum feature map.
      type: numpy.ndarray
    - spsa_steps:
      description: Number of SPSA optimization steps.
      type: int
    - C:
      description: Penalty parameter for the soft-margin support vector machine.
      type: float
  Returns:
    - aligned_kernel_parameters:
      description: The optimized 

# Invoke Quantum Kernel Alignment program

In [3]:
import numpy as np
from qka.featuremaps import FeatureMapForrelation

num_samples=5  # number of samples per class in the input data
num_features=2 # number of features in the input data
depth=2        # depth of the feature map circuit
C=1            # SVM soft-margin penalty
spsa_steps=2   # number of SPSA iterations

# Define the feature map and its initial parameters:
fm = FeatureMapForrelation(feature_dimension=num_features, depth=depth, entangler_map=None)
lambda_initial = np.random.uniform(-1,1, size=(fm._num_parameters))

# create random test data and labels:
x_train = np.random.rand(2*num_samples, num_features)
y_train = np.concatenate((-1*np.ones(num_samples), np.ones(num_samples)))

backend = provider.backend.ibmq_qasm_simulator

In [4]:
def interim_result_callback(interim_result):
    print(f"interim result: {interim_result}\n")

In [5]:
# Call QKA runtime program.
runtime_params = {
    'feature_map': fm.to_dict(),
    'data': x_train,
    'labels': y_train,
    'lambda_initial': lambda_initial,
    'spsa_steps': 2,
    'C': C
}
options = {'backend_name': backend.name()}
result = provider.runtime.run(program_name="Quantum-Kernel-Alignment",
                              options=options,
                              params=runtime_params,
                              callback=interim_result_callback,
                             ).result()


interim result: {'cost': 9.106444548790133, 'lambda': array([-0.0447036 , -0.37925792, -0.74788606, -0.17385893]), 'cost_plus': 8.799803610124869, 'cost_minus': 9.413085487455398, 'cost_final': 9.106444548790133}

interim result: {'cost': 8.806637216266033, 'lambda': array([ 0.01195016, -0.32260415, -0.80453982, -0.23051269]), 'cost_plus': 8.004876494927007, 'cost_minus': 9.60839793760506, 'cost_final': 8.806637216266033}



In [6]:
print("final result:")
print(f"aligned_kernel_parameters: {result['aligned_kernel_parameters']}")
print(f"aligned_kernel_matrix: {result['aligned_kernel_matrix']}")

final result:
aligned_kernel_parameters: [-0.00327534 -0.07018621 -0.15524259 -0.04043716]
aligned_kernel_matrix: [[1.         0.16015625 0.02148438 0.96777344 0.86816406 0.73144531
  0.88378906 0.25292969 0.72265625 0.09765625]
 [0.16015625 1.         0.23925781 0.22851562 0.39648438 0.390625
  0.45703125 0.12109375 0.57324219 0.10742188]
 [0.02148438 0.23925781 1.         0.03515625 0.0390625  0.11035156
  0.03613281 0.04296875 0.04589844 0.72851562]
 [0.96777344 0.22851562 0.03515625 1.         0.8984375  0.83398438
  0.90722656 0.26855469 0.85058594 0.05566406]
 [0.86816406 0.39648438 0.0390625  0.8984375  1.         0.96289062
  0.95703125 0.25195312 0.96972656 0.00878906]
 [0.73144531 0.390625   0.11035156 0.83398438 0.96289062 1.
  0.84960938 0.28808594 0.9140625  0.02734375]
 [0.88378906 0.45703125 0.03613281 0.90722656 0.95703125 0.84960938
  1.         0.28125    0.95410156 0.06152344]
 [0.25292969 0.12109375 0.04296875 0.26855469 0.25195312 0.28808594
  0.28125    1.        

# Test the aligned kernel

In [7]:
os.environ["NTC_PROGRAM_FILE"] = "runtime/circuit_runner.py"

In [8]:
from qka.kernel_matrix import KernelMatrix
from sklearn.svm import SVC
from sklearn import metrics

# Test the aligned kernel on test data:

x_test = np.random.rand(2*num_samples, num_features)
y_test = np.concatenate((-1*np.ones(num_samples), np.ones(num_samples)))

kernel_aligned = result['aligned_kernel_matrix']
model = SVC(C=C, kernel='precomputed')
model.fit(X=kernel_aligned, y=y_train)

km = KernelMatrix(feature_map=fm, backend=backend)
kernel_test = km.construct_kernel_matrix(x1_vec=x_test, x2_vec=x_train, parameters=result['aligned_kernel_parameters'])
labels_test = model.predict(X=kernel_test)
accuracy_test = metrics.balanced_accuracy_score(y_true=y_test, y_pred=labels_test)
print(f"accuracy test: {accuracy_test}")

/Users/jessieyu/.pyenv/versions/3.8.1/envs/qka-demo/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py:867: DeprecationWarning: The QuantumCircuit.extend() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  return self.extend(rhs)


accuracy test: 0.4
